In [ ]:
#本来是使用LSTM结构的 seq2seq 的字符级别文本生成，但是效果太差了，只能换回 transformer 的结构了

In [ ]:
# class Encoder(tf.keras.Model):
#     def __init__(self, units, tokenizer):
#         super().__init__()
#         self.tokenizer = tokenizer
#         self.vocab_size = len(tokenizer.get_vocabulary())
#         self.embedding = tf.keras.layers.Embedding(self.vocab_size, 256)
#         self.lstm = tf.keras.layers.LSTM(units, return_state=True)
        
#     def call(self, x):
#         x = self.embedding(x)
#         encoder_outputs, state_h, state_c = self.lstm(x)
#         return [state_h, state_c]

# class Decoder(tf.keras.Model):
#     def __init__(self, units, tokenizer):
#         super().__init__()
#         self.tokenizer = tokenizer
#         self.vocab_size = len(tokenizer.get_vocabulary())
#         self.embedding = tf.keras.layers.Embedding(self.vocab_size, 256)
#         self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
#         self.dense = tf.keras.layers.Dense(self.vocab_size, activation="softmax")
        
#     def call(self, x, return_state=False, training=False):
#         x, encoder_states = x
#         x = self.embedding(x)
#         x, state_h, state_c = self.lstm(x, initial_state=encoder_states)
#         x = self.dense(x)
#         if return_state:
#             return x, [state_h, state_c]
#         else:
#             return x
        
# class Seq2Seq(tf.keras.Model):
#     def __init__(self, latent_dim, input_tokenizer, target_tokenizer):
#         super().__init__()
#         self.encoder = Encoder(latent_dim, input_tokenizer)
#         self.decoder = Decoder(latent_dim, target_tokenizer)

#     def call(self, x):
#         input, target_in = x
#         encoder_outputs = self.encoder(input)
#         decoder_outputs = self.decoder((target_in, encoder_outputs))
#         return decoder_outputs

# model = Seq2Seq(latent_dim, input_tokenizer, target_tokenizer)
# def masked_loss(y_true, y_pred):
#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
#     loss = loss_fn(y_true, y_pred)
#     mask = tf.cast(y_true != 0, dtype=loss.dtype)
#     loss *= mask
#     return tf.reduce_sum(loss) / tf.reduce_sum(mask)

# def masked_acc(y_true, y_pred):
#     y_pred = tf.argmax(y_pred, axis=-1)
#     y_pred = tf.cast(y_pred, y_true.dtype)
#     match = tf.cast(tf.equal(y_true, y_pred), tf.float32)
#     mask = tf.cast(y_true != 0, tf.float32)
#     match = match * mask
#     return tf.reduce_sum(match) / tf.reduce_sum(mask)
    
# model.compile(optimizer='rmsprop', loss=masked_loss, metrics=[masked_acc])
# model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[tf.keras.callbacks.EarlyStopping(patience=)])

# def translator(sentences, max_length=100):
#     encoder_input = input_tokenizer(sentences)
#     batch_size = tf.shape(encoder_input)[0]
#     states = model.encoder(encoder_input)
#     start = tf.constant(target_tokenizer("S").numpy()[0], dtype=tf.int64)
#     start = tf.fill([batch_size, 1], start)
#     end = target_tokenizer("E").numpy()[-1]
#     done = tf.zeros([batch_size, 1], dtype=tf.bool)
#     output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
#     output_array = output_array.write(0, start)
#     for i in tf.range(max_length):
#         output = tf.transpose(output_array.stack(), perm=[1, 0, 2])
#         predictions, states = model.decoder((tf.squeeze(output, axis=-1), states), return_state=True, training=False)  # [B,1,V]   
#         predictions = predictions[:, -1:, :]
#         predicted_id = tf.argmax(predictions, axis=-1)
#         predicted_id = tf.cast(predicted_id, dtype=tf.int64) 
#         done |= predicted_id == end
#         predicted_id = tf.where(done, tf.constant(0, dtype=tf.int64), predicted_id)  # [B, 1]
#         output_array = output_array.write(i+1, predicted_id)
#         if tf.reduce_all(done):
#             break
#     output = tf.transpose(output_array.stack(), perm=[1, 0, 2])
#     output = tf.squeeze(output, axis=-1)  # Remove the last dimension
#     return output
    
# sentences = input_texts[:10]
# labels = target_texts[:10]
# output = translator(sentences)
# target_characters_i_2_c = {i:c for i,c in enumerate(target_tokenizer.get_vocabulary())}
# texts = []
# for o in output.numpy():
#     texts.append("".join([target_characters_i_2_c[i] for i in o]))
# for a,b,c in zip(sentences, texts, labels):
#     print(a, "  ", b,  "  ",c)

In [5]:
import numpy as np
import tensorflow as tf
import os 
from pathlib import Path
import tensorflow_text as tf_text
import string
import re
import pathlib
import numpy as np
import einops
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
batch_size = 64
epochs = 100
latent_dim = 1024 
data_path = "./fra-eng/fra.txt"

In [7]:
input_texts = []
target_texts = []
visited = set()
with open(data_path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        input_text, target_text = line.strip().split("\t")
        if input_text not in visited:
            visited.add(input_text)
            target_text = "[START] " + target_text + " [END]"
            input_texts.append(input_text)
            target_texts.append(target_text)
num_samples = len(input_texts)
is_train = np.random.uniform(size=(num_samples,)) < 0.9
input_texts = np.array(input_texts)
target_texts = np.array(target_texts)
 
num_samples = len(input_texts)
is_train = np.random.uniform(size=(num_samples,)) < 0.9
train_ds = tf.data.Dataset.from_tensor_slices((input_texts[is_train], target_texts[is_train])).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
val_ds = tf.data.Dataset.from_tensor_slices((input_texts[~is_train], target_texts[~is_train])).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

for input, target in train_ds.take(1):
    print(input[:3])
    print(target[:3])


def standardize(text):
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^ a-z0-9]', '')
    text = tf.strings.strip(text)
    return text
    
input_tokenizer = tf.keras.layers.TextVectorization(standardize=standardize, max_tokens=5000, ragged=True)
input_tokenizer.adapt(train_ds.map(lambda i,t:i))
target_tokenizer = tf.keras.layers.TextVectorization(standardize=standardize, max_tokens=5000, ragged=True)
target_tokenizer.adapt(train_ds.map(lambda i,t:t))

def make(input, target):
    input = input_tokenizer(input).to_tensor()
    target = target_tokenizer(target)
    target_in = target[:, :-1].to_tensor()
    target_ou = target[:, 1:].to_tensor()
    return (input, target_in), target_ou
    
train_ds = train_ds.map(make, tf.data.AUTOTUNE)
val_ds = val_ds.map(make, tf.data.AUTOTUNE)
for (input, target_in), target_ou in train_ds.take(1):
    for a,b,c in zip(input, target_in, target_ou):
        print(f"input:{a}\ntarget_in:{b}\ntarget_ou:{c}")
        break

tf.Tensor([b'Hi.' b'Run!' b'Run.'], shape=(3,), dtype=string)
tf.Tensor(
[b'[START] Salut ! [END]' b'[START] Cours ! [END]'
 b'[START] Cours ! [END]'], shape=(3,), dtype=string)
input:[2831    0]
target_in:[   2 4198    0    0    0]
target_ou:[4198    3    0    0    0]


In [11]:
print(len(input_tokenizer.get_vocabulary()), {i:c for i,c in enumerate(input_tokenizer.get_vocabulary())})

5000 {0: '', 1: '[UNK]', 2: 'i', 3: 'the', 4: 'to', 5: 'you', 6: 'a', 7: 'tom', 8: 'is', 9: 'that', 10: 'of', 11: 'he', 12: 'in', 13: 'this', 14: 'it', 15: 'do', 16: 'was', 17: 'have', 18: 'me', 19: 'my', 20: 'dont', 21: 'for', 22: 'we', 23: 'im', 24: 'what', 25: 'are', 26: 'be', 27: 'your', 28: 'not', 29: 'on', 30: 'know', 31: 'want', 32: 'like', 33: 'she', 34: 'with', 35: 'his', 36: 'can', 37: 'at', 38: 'its', 39: 'think', 40: 'were', 41: 'and', 42: 'all', 43: 'how', 44: 'did', 45: 'go', 46: 'time', 47: 'here', 48: 'about', 49: 'there', 50: 'him', 51: 'has', 52: 'very', 53: 'didnt', 54: 'cant', 55: 'going', 56: 'get', 57: 'will', 58: 'no', 59: 'as', 60: 'they', 61: 'had', 62: 'youre', 63: 'mary', 64: 'her', 65: 'up', 66: 'out', 67: 'one', 68: 'ill', 69: 'need', 70: 'just', 71: 'an', 72: 'if', 73: 'good', 74: 'ive', 75: 'why', 76: 'thats', 77: 'would', 78: 'from', 79: 'so', 80: 'when', 81: 'by', 82: 'really', 83: 'who', 84: 'us', 85: 'never', 86: 'help', 87: 'been', 88: 'got', 89: 'se

In [13]:
print(print(len(target_tokenizer.get_vocabulary())),{i:c for i,c in enumerate(target_tokenizer.get_vocabulary())})

5000
None {0: '', 1: '[UNK]', 2: 'start', 3: 'end', 4: 'je', 5: 'de', 6: 'a', 7: 'pas', 8: 'que', 9: 'la', 10: 'le', 11: 'tom', 12: 'ne', 13: 'il', 14: 'est', 15: 'un', 16: 'ce', 17: 'nous', 18: 'tu', 19: 'les', 20: 'une', 21: 'jai', 22: 'vous', 23: 'en', 24: 'pour', 25: 'cest', 26: 'me', 27: 'suis', 28: 'ca', 29: 'faire', 30: 'des', 31: 'dans', 32: 'plus', 33: 'du', 34: 'elle', 35: 'tout', 36: 'fait', 37: 'ma', 38: 'mon', 39: 'qui', 40: 'sur', 41: 'au', 42: 'se', 43: 'et', 44: 'te', 45: 'avec', 46: 'veux', 47: 'quil', 48: 'cette', 49: 'sont', 50: 'pense', 51: 'si', 52: 'nest', 53: 'y', 54: 'son', 55: 'etait', 56: 'tres', 57: 'dit', 58: 'temps', 59: 'ou', 60: 'cela', 61: 'nai', 62: 'sais', 63: 'peux', 64: 'moi', 65: 'ete', 66: 'ils', 67: 'etre', 68: 'ici', 69: 'jamais', 70: 'chose', 71: 'beaucoup', 72: 'pourquoi', 73: 'bien', 74: 'lui', 75: 'tous', 76: 'on', 77: 'vraiment', 78: 'sa', 79: 'votre', 80: 'personne', 81: 'besoin', 82: 'estce', 83: 'par', 84: 'quelque', 85: 'ton', 86: 'comme

In [50]:
UNITS = 1024
class Encoder(tf.keras.layers.Layer):
    def __init__(self, text_processor, units):
        super(Encoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.units = units
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, units, mask_zero=True)
        self.rnn = tf.keras.layers.Bidirectional(merge_mode='sum', layer=tf.keras.layers.GRU(units, return_sequences=True, recurrent_initializer='glorot_uniform'))
    
    def call(self, x):
        x = self.embedding(x)
        x = self.rnn(x)
        return x
    
    def convert_input(self, texts):
        texts = tf.convert_to_tensor(texts)
        if len(texts.shape) == 0:
            texts = tf.convert_to_tensor(texts)[tf.newaxis]
        context = self.text_processor(texts).to_tensor()
        context = self(context)
        return context
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=2, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x, context):
        attn_ouput, attn_scores = self.mha(query=x, value=context, return_attention_scores=True)
        attn_scores = tf.reduce_mean(attn_scores, axis=1)
        self.last_attention_weights = attn_scores
        x = self.add([x, attn_ouput])
        x = self.layernorm(x)
        return x
class Decoder(tf.keras.layers.Layer):
    def __init__(self, text_processor, units):
        super(Decoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.word_to_id = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')
        self.units = units
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, units, mask_zero=True)
        self.rnn = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.attention = CrossAttention(units)
        self.output_layer = tf.keras.layers.Dense(self.vocab_size)
        
    def call(self, context, x, state=None, return_state=False):
        x = self.embedding(x)
        x, state = self.rnn(x, initial_state=state)
        x = self.attention(x, context)
        self.last_attention_weights = self.attention.last_attention_weights
        logits = self.output_layer(x)
        if return_state:
            return logits, state
        return logits
    
    def get_initial_state(self, context):
        batch_size = tf.shape(context)[0]
        start_tokens = tf.fill([batch_size, 1], self.start_token)
        done = tf.zeros([batch_size, 1], dtype=tf.bool)
        embedded = self.embedding(start_tokens)
        return start_tokens, done, self.rnn.get_initial_state(embedded)[0]  # [batch_size, units]
    
    def tokens_to_text(self, tokens):
        words = self.id_to_word(tokens)
        result = tf.strings.reduce_join(words, axis=-1, separator=' ')
        result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
        result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
        return result
    
    def get_next_token(self, context, next_token, done, state,temperature=0.0):
        logits, state = self(context, next_token, state=state, return_state=True)
        if temperature == 0.0:
            next_token = tf.argmax(logits, axis=-1)
        else:
            logits = logits[:, -1, :]/temperature
            next_token = tf.random.categorical(logits, num_samples=1)
        done |= (next_token == self.end_token)
        next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)
        return next_token, done, state

class Translator(tf.keras.Model):
    def __init__(self, units, context_text_processor, target_text_processor):
        super().__init__()
        self.encoder = Encoder(context_text_processor, units)
        self.decoder = Decoder(target_text_processor, units)
    
    def call(self, inputs):
        context, x = inputs
        context = self.encoder(context)
        logits = self.decoder(context, x)
        try:
            del logits._keras_mask
        except AttributeError:
            pass
        return logits
    
    def translate(self, texts, max_length=500, temperature=tf.constant(0.0)):
        context = self.encoder.convert_input(texts)
        next_token, done, state = self.decoder.get_initial_state(context)
        tokens = tf.TensorArray(tf.int64, size=1, dynamic_size=True)
        for t in tf.range(max_length):
            next_token, done, state = self.decoder.get_next_token(context, next_token, done, state, temperature)
            tokens = tokens.write(t, next_token)
            if tf.reduce_all(done):
                break
        tokens = tokens.stack()
        tokens = einops.rearrange(tokens, 't batch 1 -> batch t')
        text = self.decoder.tokens_to_text(tokens)
        return text
def masked_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)
    mask = tf.cast(y_true!=0, dtype=loss.dtype)
    loss *= mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true !=0 ,tf.float32)
    return tf.reduce_sum(match) / tf.reduce_sum(mask)

model = Translator(UNITS, input_tokenizer, target_tokenizer)
model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])


In [52]:
history = model.fit(train_ds, epochs=100,validation_data=val_ds, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
2343/2343 [==============================] - 39s 15ms/step - loss: 2.4390 - masked_acc: 0.5517 - masked_loss: 2.4397 - val_loss: 2.3819 - val_masked_acc: 0.5317 - val_masked_loss: 2.3833
Epoch 2/100
2343/2343 [==============================] - 49s 21ms/step - loss: 1.4226 - masked_acc: 0.6830 - masked_loss: 1.4234 - val_loss: 1.9670 - val_masked_acc: 0.5712 - val_masked_loss: 1.9693
Epoch 3/100
2343/2343 [==============================] - 49s 21ms/step - loss: 1.1937 - masked_acc: 0.7192 - masked_loss: 1.1945 - val_loss: 1.8819 - val_masked_acc: 0.5859 - val_masked_loss: 1.8842
Epoch 4/100
2343/2343 [==============================] - 49s 21ms/step - loss: 1.0566 - masked_acc: 0.7420 - masked_loss: 1.0574 - val_loss: 1.8392 - val_masked_acc: 0.5941 - val_masked_loss: 1.8422
Epoch 5/100
2343/2343 [==============================] - 49s 21ms/step - loss: 0.9599 - masked_acc: 0.7590 - masked_loss: 0.9606 - val_loss: 1.7605 - val_masked_acc: 0.6024 - val_masked_loss: 1.7636
Epoch

In [ ]:
result = model.translate(input_texts[:20]).numpy()
for a,b,c in zip(input_texts[:20], result, target_texts[:20]):
    print(a,"预测---->" ,b.decode("utf-8").split("end")[0], "标签---->" ,c) # 效果不太好是因为一方面没有完全用注意力机制的 seq2seq ，另一方面就是法语在处理的时候没有进行很好的分词